<img src="https://raw.githubusercontent.com/alan-barzilay/NLPortugues/master/imagens/logo_nlportugues.png"   width="150" align="right">


# Lista 5 - Vanishing & Exploding Gradient



______________



Nessa lista exploraremos alguns problemas que podemos encontrar ao treinarmos uma rede recorrente. Esses problemas não são únicos das redes recorrentes, qualquer rede profunda pode sofrer de vanishing e exploding gradient mas as redes recorrentes são especialmente instaveis devido a utilização da mesma matriz de pesos repetidas vezes.

Começaremos explorando o exploding gradient e alguns de seus sintomas, em seguida utilizaremos gradient cliping para combate-lo.
Por fim estudaremos uma rede que sofre de vanishing gradient e utilizaremos o [TensorBoard](https://github.com/tensorflow/tensorboard) para visualizar os gradientes e pesos da rede para entender melhor sua dinâmica. Exploraremos tambem algumas maneiras de combater o vanishing gradient e como elas alteram a dinâmica da rede através do TensorBoard.



**Nota:** Nesta aula utilizaremos um callback especial para visualizarmos os gradientes e pesos da rede. Para garantir que tudo funcione corretamente utilizaremos uma versão mais antiga do tensorflow já que a versão 2.3.0 introduziu mudanças que quebram esse callback.

In [1]:
%pip install 'tensorflow<2.3' --force-reinstall

ERROR: Could not find a version that satisfies the requirement tensorflow<2.3 (from versions: 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.0.post1, 2.15.1, 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0)
ERROR: No matching distribution found for tensorflow<2.3


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

from sklearn.datasets import make_circles
from numpy import where
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.initializers import RandomUniform

In [3]:
tf.__version__

'2.17.1'



# Exploding

Para essa parte da lista nós preparamos uma rede, note como a *loss* cresce exponencialmente até virar infinita e logo em seguida NaN. Esse é um sintoma classico de exploding gradient. O gradiente está tão elevado que a cada etapa de backpropagation o passo de atualização dos parametros leva a um aumento na *loss* e isso segue crescendo até que exploda.


In [4]:
def f1(x):
    return 5+ 10*x

xs = [x for x in range(100)]
ys = [f1(x) for x in range(100)]


In [5]:
opt = keras.optimizers.SGD()
model = tf.keras.Sequential([keras.layers.Dense(units=1, input_shape=[1])])
model.compile(optimizer=opt, loss="mean_squared_error")
model.fit(xs,ys,epochs=400)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: Unrecognized data type: x=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99] (of type <class 'list'>)

## Gradient Cliping
## <font color='blue'>Questão 1 </font>


Utilizando a mesma arquitetura, realize gradient clipping no otimizador para contornar o problema de exploding gradient, compile e treine seu novo modelo.
Você pode usar tanto o parametro `clipvalue` quanto `clipnorm` desde que sua rede consiga minimizar a loss.


In [ ]:

model = tf.keras.Sequential([keras.layers.Dense(units=1, input_shape=[1])])

# Seu código aqui

model.fit(xs,ys,epochs=400)

________________________



# Vanishing

Lidar com Vanishing Gradient é muito mais desafiador do que com exploding gradient. Não é trivial determinar se o baixo desempenho de sua rede é causado por vanishing gradient uma vez que seus sintomas são relativamente genéricos e ele pode ser apenas mais um dos fatores que prejudicam seu desempenho. Além disso não existe uma solução geral e definitiva como o gradient cliping em casos de explosão do gradiente.

Preparamos algumas redes para poder explorar um caso mais simples de vanishing gradient e também uma possivel solução. Começamos gerando um dataset simples de classificação e treinamos uma rede rasa que obtem uma boa performance.
Ao aprofundarmos essa rede podemos notar que sua performance cai drasticamente se tornando quase tão eficiente quanto jogar uma moeda para chutar a classe do ponto, ela nem mesmo é capaz de "*overfittar*" os dados. Utilizaremos o TensorBoard para explorar os gradientes dessa rede mais profunda e tentar analisar essa queda de performance. Em seguida utilizaremos uma nova forma de inicialização dos pesos da rede para tentar recuperar nossa performance e novamente inspecionar os seus gradientes para tentar ganhar algum insight do que está acontecendo dentro dela.

___________

## TensorBoard e Callbacks

O TensorBoard é uma ferramenta de monitoramento e visualização de redes neurais. Nós o utilizaremos para visualizar os gradientes de nossa rede e poder entender mais afundo o que está causando nosso fraco desempenho. Para mais informações sobre o TensorBoard recomendamos este [guia da documentação oficial](https://www.tensorflow.org/tensorboard/get_started). O TensorBoard pode ser iniciado como parte do notebook usando a "magica"  `%load_ext tensorboard` para carrega-lo e `%tensorboard --logdir logs/` para inicia-lo. Outra maneira de utiliza-lo é independente pelo terminal a partir dos logs salvos na pasta `log_dir` com o comando `tensorboard --logdir="logs/"`.

Para utilizar o TensorBoard nós utilizamos um callback ao treinar nossa rede.

[Callbacks](https://keras.io/api/callbacks/#tensorboard) são objetos do Keras capazes de realizar ações em diferentes etapas do treinamento, como a cada final de epoch ou antes de cada mini-batch. Três exemplos interessates de callbacks são o [EarlyStopping](https://keras.io/api/callbacks/early_stopping/) que te permite encerrar o treinamento de sua rede quando você atinge uma performance minima desejada, o [ModelCheckpoint](https://keras.io/api/callbacks/model_checkpoint/) que lhe permite criar checkpoints do seu modelo para poder reiniciar o treinamento sem perder todo seu progresso em caso de algum problema (como seus gradientes explodindo por exemplo) e o [LearningRateScheduler](https://keras.io/api/callbacks/learning_rate_scheduler/) que lhe permite alterar o learning rate do seu otimizador conforme a epoch de treinamento.

O TensorBoard possui um callback que nos permite monitorar o desempenho e parâmetros de nosso modelo ao longo do treinamento, porém para guardar os gradientes da rede necessitamos expandir e adaptar o [callback padrão do TensorBoard](https://keras.io/api/callbacks/tensorboard/).

Nós ja implementamos para vocês uma extensão desse callback, para usar este callback basta importa-lo e declara-lo da seguinte maneira:


```python
log_dir = "logs/"
tensorboard_callback = ExtendedTensorBoard(x=dados_treino, y=labels_treino,log_dir=log_dir,histogram_freq=1)
```
________________

In [ ]:
from tensorflow.keras.callbacks import TensorBoard
%load_ext tensorboard

class ExtendedTensorBoard(TensorBoard):
    """
    Adaptado de https://github.com/tensorflow/tensorflow/issues/31542
    """
    def __init__(self, x, y,log_dir='logs',
                            histogram_freq=0,
                            write_graph=True,
                            write_images=False,
                            update_freq='epoch',
                            profile_batch=2,
                            embeddings_freq=0,
                            embeddings_metadata=None,
                            **kwargs,):
        self.x=x
        self.y=y
        super(ExtendedTensorBoard, self).__init__(log_dir,
                                                    histogram_freq,
                                                    write_graph,
                                                    write_images,
                                                    update_freq,
                                                    profile_batch,
                                                    embeddings_freq,
                                                    embeddings_metadata,)

    def _log_gradients(self, epoch):
        writer = self._get_writer(self._train_run_name)
        with writer.as_default(), tf.GradientTape() as g:

            features=tf.constant(self.x)
            y_true=tf.constant(self.y)

            y_pred = self.model(features)  # forward-propagation
            loss = self.model.compiled_loss(y_true=y_true, y_pred=y_pred)  # calculate loss
            gradients = g.gradient(loss, self.model.trainable_weights)  # back-propagation

            # In eager mode, grads does not have name, so we get names from model.trainable_weights
            for weights, grads in zip(self.model.trainable_weights, gradients):
                tf.summary.histogram(
                    weights.name.replace(':', '_') + '_grads', data=grads, step=epoch)

        writer.flush()

    def on_epoch_end(self, epoch, logs=None):
#         Sobre-escrevemos essa função da super classe pois necessitamos
#         adicionar a funcionalidade de gravar os gradientes.
#         Como tambem queremos suas funcionalidades originais, tambem invocamos o metodo super
        super(ExtendedTensorBoard, self).on_epoch_end(epoch, logs=logs)

        if self.histogram_freq and epoch % self.histogram_freq == 0:
            self._log_gradients(epoch)


## Definindo nossos dados
Primeiro definiremos um toy dataset bem simples que utilizaremos para nossos modelos e uma função auxiliar para facilitar a comparação de nossos modelos.


Esses dados e redes foram inspirados e adaptados [deste post](https://machinelearningmastery.com/how-to-fix-vanishing-gradients-using-the-rectified-linear-activation-function/).


In [ ]:
# gera dataset de classificação
X, y = make_circles(n_samples=1000, noise=0.1, random_state=1)

# escala input para [-1,1]
scaler = MinMaxScaler(feature_range=(-1, 1))
X = scaler.fit_transform(X)

# plota visualização do dataset
for i in range(2):
    samples_ix = where(y == i)
    plt.scatter(X[samples_ix, 0], X[samples_ix, 1], label=str(i))
plt.legend()
plt.show()

# separa em teste e treino
n_train = 500
trainX, testX = X[:n_train, :], X[n_train:, :]
trainy, testy = y[:n_train], y[n_train:]

In [ ]:
def run_model(model,log_to_tb= False ,trainX=trainX,trainy=trainy,testX=testX,testy=testy):
    """
    Função auxiliar que recebe um modelo e realiza seu treinamento e avaliação no dataset.
    """
    model.summary()

    # compila modelo
    opt = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

    #Cria log do modelo pra visualizarmos no TensorBoard se a flag estiver ligada
    callbacks = None
    if log_to_tb==True:
        log_dir = "logs/" + model.name
        callbacks=[ExtendedTensorBoard(x=trainX, y=trainy,log_dir=log_dir,histogram_freq=1)]


    # fit modelo
    history = model.fit(trainX, trainy, validation_data=(testX, testy), epochs=500, verbose=0, callbacks=callbacks)

    # avalia modelo
    _, train_acc = model.evaluate(trainX, trainy, verbose=0)
    _, test_acc = model.evaluate(testX, testy, verbose=0)
    print("\n")
    print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))


    # plota acurácia/training history
    plt.ylim(0, 1)
    plt.title("Acurácia "+ model.name)
    plt.plot(history.history['accuracy'], label='train')
    plt.plot(history.history['val_accuracy'], label='test')
    plt.legend()
    plt.show()

## Rede rasa
Aqui temos uma rede rasa com apenas uma camada oculta e uma de output, note que ela é capaz de atingir uma performance razoavel após 300 epochs.

Nos estamos utilizando um inicializador diferente do padrão para os pesos da camada, ele retirar os pesos iniciais a partir de uma distribuição uniforme no intervalo [0,1].

In [ ]:
#define modelo raso
init = RandomUniform(minval=0, maxval=1)

model = keras.Sequential(name="modelo_raso")
model.add(keras.layers.Dense(5,
                       input_dim=2,
                       activation="tanh",
                       kernel_initializer=init,
                       name="raso_1"))
model.add(keras.layers.Dense(1,
                       activation='sigmoid',
                       kernel_initializer=init,
                       name="raso_output"))


run_model(model)

## Rede funda

Agora tornaremos nossa rede mais funda com 5 camadas ocultas e uma de output, note como a performance cai drasticamente e se torna próxima a um chute aleatório. Embora o modelo seja mais complexo e poderoso nós não conseguimos treina-lo.

In [ ]:
# define modelo mais fundo
init = RandomUniform(minval=0, maxval=1)

model = Sequential(name="modelo_fundo")
model.add(Dense(5, input_dim=2, activation='tanh', kernel_initializer=init,name="funda_1"))
model.add(Dense(5, activation='tanh', kernel_initializer=init,name="funda_2"))
model.add(Dense(5, activation='tanh', kernel_initializer=init,name="funda_3"))
model.add(Dense(5, activation='tanh', kernel_initializer=init,name="funda_4"))
model.add(Dense(5, activation='tanh', kernel_initializer=init,name="funda_5"))
model.add(Dense(1, activation='sigmoid', kernel_initializer=init,name="funda_output"))


run_model(model,log_to_tb=True)

### Utilizando inicialização de Xavier Glorot

Agora utilizaremos uma técnica de combate ao vanishing gradient, utilizaremos outro inicializador para os pesos da rede. O [inicializador de Xavier Glorot](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf) é o inicializador default de algumas camadas do keras como a camada densa que utilizamos.



In [ ]:
# define modelo mais fundo com inicializador de pesos melhor
model = Sequential(name="modelo_xavier")
model.add(Dense(5, input_dim=2, activation='tanh',kernel_initializer="glorot_uniform", name="xavier_1"))
model.add(Dense(5, activation='tanh',kernel_initializer="glorot_uniform", name="xavier_2"))
model.add(Dense(5, activation='tanh',kernel_initializer="glorot_uniform", name="xavier_3"))
model.add(Dense(5, activation='tanh', kernel_initializer="glorot_uniform",name="xavier_4"))
model.add(Dense(5, activation='tanh',kernel_initializer="glorot_uniform", name="xavier_5"))
model.add(Dense(1, activation='sigmoid', kernel_initializer="glorot_uniform", name="xavier_output"))

run_model(model,log_to_tb=True)

## <font color='blue'>Questão 2 </font>
Inicialize o TensorBoard e cheque os histogramas e as distribuições dos valores dos gradientes das 2 redes, como eles se diferem de uma rede para outra? Preste atenção em particular para a diferença de amplitude dos seus valores. Que conclusão podemos tirar? Insira/cole imagens do TensorBoard para basear seus argumentos, pode ser uma simples captura de tela.

**Obs:** Você pode filtrar elementos ao escrever a tag "kernel_0_grads" como filtro para facilitar sua exploração dos gradientes dos pesos excluindo os termos de bias/viés.

**Obs2:** Lembre-se de apagar a pasta `logs` se você for retreinar alguma das redes, caso contrário ele irá dar um append nos seus 2 treinamentos e seus graficos de loss e acurácia ficarão errados com loops ligando o começo de um treinamento ao final do outro.

**<font color='red'> Sua resposta aqui </font>**

In [ ]:
%tensorboard --logdir logs